# 簡介

Hinton等人表示，人工神經網絡不應當追求"活動中的視角不變性"(即使用單一的標量輸出，來總結一個局部池中的重複特徵檢測器的活動)，而應當使用局部的"膠囊"。每個膠囊，學習辨識一個有限的觀察條件，和變形範圍內，隱式定義的視覺實體，並輸出實體在有限範圍內，存在的機率，及一組"實例參數"，這些實例參數，可能包括實體的姿勢、照明條件，以及變形訊息。

當膠囊工作正常時，視覺實體存在的機率，具有**局部不變性**: 當時體在膠囊覆蓋的範圍內的外觀流形上，進行移動時，其機率不會改變。

而實例參數，是**等變的**: 即隨著觀察條件的變化，實體在外觀流形上，進行移動時，實例參數也會相應地變化。因為實例參數是，實體在外觀流形上的內在座標。

In [ ]:
# 掛載雲端硬碟

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


詳見[此處](https://blog.csdn.net/m0_46384757/article/details/121559514?spm=1001.2101.3001.6661.1&utm_medium=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-121559514-blog-79763486.pc_relevant_multi_platform_whitelistv4&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-121559514-blog-79763486.pc_relevant_multi_platform_whitelistv4&utm_relevant_index=1)。

# 膠囊網絡

當我們通過計算機圖形渲染(設計)，來建構對象時，我們需要指定並提供一些幾何信息，比如，告訴計算機:
* 在何處繪製對象
* 該對象的比例
* 角度及其他空間信息

其中的對象，通常是通過參數設置來呈現的。當這些信息全部表示出來，就是屏幕上的一個對象。

但是，如果只是透過觀察照片中的物體，來提取信息，則**膠囊網絡(Capsule Network)**提出一個思想: **逆渲染(Inverse redering)**。

在膠囊網絡中，網絡要學習如何反向渲染圖象: 通過觀察圖像，嘗試預測圖象的實例參數。

膠囊網絡通過重現它檢測到的對象，將重現結果與訓練數據中的標記，進行比較，來學習如何預測。通過反覆地學習，它將可以實現較準確的實例參數預測。

## 膠囊

一個膠囊網絡，是由膠囊，而不是神經元構成的。它將個體神經元，替換成了一組神經元組成的向量，這些神經元被包裹在一起，就組成了膠囊。它們可以學習在一張圖片中，在一定區域內，檢查一個特定的對象。輸出是一個向量。

其中，每個向量的長度，代表了物體是否存在的機率。而每個向量的方向，記錄了物體的姿態參數。如果物體有稍微的變化(移動、旋轉、尺寸變化等)，則膠囊也會輸出一個，長度相同，但方向稍微變化的向量。因此，膠囊是"等變的"。

在傳統神經網絡裡，一個神經元，一般會進行如下的操作:
1. 輸入標量的標量加權
2. 對加權後的標量，進行求和
3. 對結果進行非線性變換，生成新的標量

而膠囊可以理解為，上述步驟的向量版，並增加對輸入的仿射變換:
1. 輸入向量的乘法:
 * 膠囊的輸入向量，編碼了低層膠囊檢測出的相應對像的機率，
 * 而向量的方向，編碼了檢測出的對象的一些內部狀態
2. 仿射變換:
 * 將這些像量，乘以相應的權重矩陣W
 * W編碼了低層特徵(眼睛、嘴巴)和高層特徵(臉)之間的空間關係，以及其他重要關係
3. 動態路由:
 * 使用權重，決定來自下一層的像量，將會如何進入高一層的向量，這一過程，不加入偏移項
 * 動態路由的關鍵在於，權重係數該如何確定。對於最大池化而言，只有一個值能夠進入本層，其效果就是一個獨熱向量。而動態路由通過一種類似具類的方式，對向量賦予權重
4. 輸出的非線性變換:
 * 膠囊網絡的非線性啟動函數，接收一個向量，然後在不改變方向的前提下，將其長度壓縮到1以下


## CNN的缺陷

膠囊網絡的目的，是要解決CNN的一個缺陷: 它只會觀察圖像的特徵，而不會關注特徵的姿態。

這主要的原因在於，CNN中的pooling會丟失空間中的大部分信息。Geoffrey Hinton實際上認為: 在CNN中應用pooling，是一個很大的錯誤，它工作得很好是事實，是一場災難。

## 解決方法

膠囊網絡的解決方法是，實現對空間信息進行編碼的同時，也計算物體的存在機率。

這可以用向量來表示: 
* 模: 特徵的存在機率
* 方向: 特徵的姿態信息

此外，也可以透過自動編碼器的"重建損失"，來改善。

## 動態路由

在膠囊網絡中，權重參數並不是通過訓練得到的，而是由**動態路由**決定的。

在這裡的迭代計算過程中，在係數最終確定前的步驟，是不會計算梯度的，而單純通過"聚類"的方式得到，並不會參與梯度下降的調整參數過程。

它會使用softmax，表示輸入向量進入每一個高層膠囊的機率。即，向量在上層的某個膠囊空間中，越接近中心簇，則該膠囊感知到的可能性越大，因此分配到這個膠囊的權重就越大。

在論文中，列出了動態路由的實際算法:
1. 假設該路由演算法，發生在網絡的第k層，i為膠囊單元的個數，j為每個膠囊單元向量的維數，則輸入值$\hat u_{j|i}$是主膠囊網絡的輸出特徵$u_i$與權重$w_{ij}$的乘積。
2. 初始化變數$b_{ij}$，使其為0。
 * 它與耦合係數c，具有相同的長度
 * 在迭代時，c就是由b做softmax計算得來。隨著算法的繼續，分布將會改變
3. 讓路由演算法，按照指定的迭代次數r，進行迭代
4. 對變數b做softmax操作，獲得耦合係數c。它是每個權重的機率
 * b與c的個數，與膠囊單元的個數相同
5. 將c與$\hat u_{j|i}$相乘，並將乘機的結果，進行相加，得到數字膠囊(k+1層)的輸出向量s
6. 透過啟動函數squash，對s做非線性變換，獲得輸出$v_j$。這確保了向量的方向被保存下來，而長度被限制在1以下
7. 將$v_j$與$\hat u_{j|i}$進行點積運算，再與b相加，得到新的b值
 * 點積的作用是: 計算膠囊的輸入和膠囊的輸出之相似度，利用相似度，來更新b值
8. 該路由演算法，需要迭代計算r次。
 * 在進行路由更新的同時，也更新了最後的輸出結果$v_j$
 * 在迭代結束後，將最後的$v_j$傳回，計算損失與輸出，並確定路由的權重

在路由(Routing)過程中，下層膠囊，將輸入向量，傳送到上層膠囊。

對於可以路由到的每個更高層膠囊，下層膠囊通過將自己的輸出，乘上權重矩陣，來計算預測向量。如果預測向量與上層的輸出，具有較大的標量積，則存在從頂向下的反饋，其具有增加上層膠囊耦合係數，減小其他膠囊耦合係數的效果。

就類似聚類，並剔除離群點。

## squash啟動函數

因為膠囊網絡的輸出是向量，所以有必要為膠囊網絡，設計一套全新的啟動函數:
$$y=\frac{||x||^2}{1+||x||^2}\frac{x}{||x||}$$
其中
* 前項: 將數值轉換成[0,1]之間
* 後項: 保留原有向量的方向

兩者結合，使值域在[-1,1]。

如果拋開理論，單純從輸入、輸出的數值上看，則squash與一般的啟動函數沒什麼區別，把它換成一般的啟動函數，也能夠執行。只不過大量的實驗表明，squash在膠囊網絡中的表現，確實比其他啟動函數要好。

## 損失函數

膠囊網絡的損失函數，由兩部分組成:
* 為了應付重疊數字識別任務，因為目標可能包含多個標籤，也可能只有一個標籤，所以使用"邊距損失"，用於懲罰偽陽性與偽陰性的識別結果
* 為了能夠保留特徵的空間信息，設置一個圖像的重構任務，並設置"重建損失"，即使用一個解碼器，進行還原，並使用MSE損失，與原圖像進行像素級別地比較，並計算損失(用於計算空間信息、向量方向)

對於k個類別的數字，我們希望最高層的膠囊，若且維若，一個數字出現在圖象中時，具有一個長的實例化向量。為了允許多個數字，對於每一個膠囊，使用獨立的損失函數:

1. **邊距損失(margin loss)**是一種最大化正負樣本到超平面距離的演算法:
$$L_k=T_kmax(0, m^+-||V_k||)^2+\lambda(1-T_k)max(0, ||V_k||-m^-)^2$$
其中
* $L_k$ : 損失值
* $T_k$ : 標籤
* $:m^+$ : 最大值的錨點
* $:m^-$ : 最小值的錨點
* $V_k$ : 模型輸出的預測值
* $\lambda$ : 縮放參數
* $||V_k||$ : $V_k$的範數

然後將所有損失值取平均，就得到最終的損失。

以MNIST為例:
* 假設$m^+=0.9、m^-=0.1、\lambda=0.5$(確保訓練中，數值的穩定性)，因為輸出值的形狀是[10, 16]，所以$L_k$的形狀是[10, 16]
* 對每個類別的16維向量，進行相加，使其形狀變成[10, 1]
* 再取平均值，就得到loss值

2. **MSE損失**就是常見的回歸任務的損失。

## 優點

1. 膠囊網絡可以使用少很多的訓練數據，就能夠在MNIST數聚集上，達到業界最佳精度
2. 膠囊網絡可以很好地應對**模糊性**，非常適合擁擠的場景
3. 在膠囊網絡中，細節的姿態信息，會被網絡保存下來，輸入上的微小變化，會帶來輸出上的小變化，這讓它能使用一個簡單和統一的架構，來應對不同的視覺任務，非常適合目標檢測任務
4. 膠囊網絡的路由樹結構，映射著對象組件的層次結構關係，可以告知我們這些布建的層次結構

## 缺點

1. 相較於CNN，膠囊網絡的訓練速度很慢，動態路由過程，非常耗運算資源
2. 因為無論是什麼類型的對象，在給定的位置，永遠只有一個膠囊，所以膠囊網絡不可能檢測出，兩個非常靠近的同一類型的對象，稱為**擁擠現象**
3. 雖然它可以在MNIST上，表現出很好的性能，但是並未在較大的如ImageNet上，獲得好的結果。這是因為在圖象中，發現的信息過多，會使膠囊脫落

總體來看，膠囊網絡仍然處於研究和開發階段，並且不夠可靠，還沒有很成熟的任務。

## 模型架構

低層的膠囊，用於檢測一些特定模式的出現機率與姿態，而高層的膠囊，用於檢測更加複雜的圖像。

整個模型架構，主要分為三層:
1. 卷積層: 用於抽取底層特徵
2. 初始膠囊層: 用於第二次捲積，並初始化膠囊輸入
3. 路由膠囊層: 用於編碼空間信息，並進行最終分類

# 實作

## 載入套件

In [ ]:
import torch
from torchvision import datasets, transforms

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
from tqdm import tqdm

USE_CUDA = True if torch.cuda.is_available() else False
USE_CUDA

True

## 建立資料集

In [ ]:
class Dataset:
    def __init__(self, dataset, _batch_size):
        super(Dataset, self).__init__()
        if dataset == 'mnist':
            dataset_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])

            train_dataset = datasets.MNIST('/data/mnist', train=True, download=True,
                                           transform=dataset_transform)
            test_dataset = datasets.MNIST('/data/mnist', train=False, download=True,
                                          transform=dataset_transform)

            self.train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=_batch_size, shuffle=True)
            self.test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=_batch_size, shuffle=False)

        elif dataset == 'cifar10':
            data_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
            train_dataset = datasets.CIFAR10(
                '/data/cifar', train=True, download=True, transform=data_transform)
            test_dataset = datasets.CIFAR10(
                '/data/cifar', train=False, download=True, transform=data_transform)

            self.train_loader = torch.utils.data.DataLoader(
                train_dataset, batch_size=_batch_size, shuffle=True)

            self.test_loader = torch.utils.data.DataLoader(
                test_dataset, batch_size=_batch_size, shuffle=False)
        elif dataset == 'office-caltech':
            pass
        elif dataset == 'office31':
            pass

## 建立模型

### 卷積層

In [ ]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, out_channels=256, kernel_size=9):
        super(ConvLayer, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels,
                              kernel_size=kernel_size,
                              stride=1
                              )

    def forward(self, x):
        return F.relu(self.conv(x))

### 初始膠囊層

In [ ]:
class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32, kernel_size=9, num_routes=32 * 6 * 6):
        super(PrimaryCaps, self).__init__()
        self.num_routes = num_routes
        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=2, padding=0)
            for _ in range(num_capsules)])

    def forward(self, x):
        u = [capsule(x) for capsule in self.capsules]
        u = torch.stack(u, dim=1)
        u = u.view(x.size(0), self.num_routes, -1)
        return self.squash(u)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm * input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

### 路由膠囊層

In [ ]:
class DigitCaps(nn.Module):
    def __init__(self, num_capsules=10, num_routes=32 * 6 * 6, in_channels=8, out_channels=16):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules

        self.W = nn.Parameter(torch.randn(1, num_routes, num_capsules, out_channels, in_channels))

    def forward(self, x):
        batch_size = x.size(0)
        x = torch.stack([x] * self.num_capsules, dim=2).unsqueeze(4)

        W = torch.cat([self.W] * batch_size, dim=0)
        u_hat = torch.matmul(W, x)

        b_ij = Variable(torch.zeros(1, self.num_routes, self.num_capsules, 1))
        if USE_CUDA:
            b_ij = b_ij.cuda()

        num_iterations = 3
        for iteration in range(num_iterations):
            c_ij = F.softmax(b_ij, dim=1)
            c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(4)

            s_j = (c_ij * u_hat).sum(dim=1, keepdim=True)
            v_j = self.squash(s_j)

            if iteration < num_iterations - 1:
                a_ij = torch.matmul(u_hat.transpose(3, 4), torch.cat([v_j] * self.num_routes, dim=1))
                b_ij = b_ij + a_ij.squeeze(4).mean(dim=0, keepdim=True)

        return v_j.squeeze(1)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm * input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

### 解碼器

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_width=28, input_height=28, input_channel=1):
        super(Decoder, self).__init__()
        self.input_width = input_width
        self.input_height = input_height
        self.input_channel = input_channel
        self.reconstraction_layers = nn.Sequential(
            nn.Linear(16 * 10, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, self.input_height * self.input_width * self.input_channel),
            nn.Sigmoid()
        )

    def forward(self, x, data):
        classes = torch.sqrt((x ** 2).sum(2))
        classes = F.softmax(classes, dim=0)

        _, max_length_indices = classes.max(dim=1)
        masked = Variable(torch.sparse.torch.eye(10))
        if USE_CUDA:
            masked = masked.cuda()
        masked = masked.index_select(dim=0, index=Variable(max_length_indices.squeeze(1).data))
        t = (x * masked[:, :, None, None]).view(x.size(0), -1)
        reconstructions = self.reconstraction_layers(t)
        reconstructions = reconstructions.view(-1, self.input_channel, self.input_width, self.input_height)
        return reconstructions, masked

### 膠囊網絡

In [ ]:
class CapsNet(nn.Module):
    def __init__(self, config=None):
        super(CapsNet, self).__init__()
        if config:
            self.conv_layer = ConvLayer(config.cnn_in_channels, config.cnn_out_channels, config.cnn_kernel_size)
            self.primary_capsules = PrimaryCaps(config.pc_num_capsules, config.pc_in_channels, config.pc_out_channels,
                                                config.pc_kernel_size, config.pc_num_routes)
            self.digit_capsules = DigitCaps(config.dc_num_capsules, config.dc_num_routes, config.dc_in_channels,
                                            config.dc_out_channels)
            self.decoder = Decoder(config.input_width, config.input_height, config.cnn_in_channels)
        else:
            self.conv_layer = ConvLayer()
            self.primary_capsules = PrimaryCaps()
            self.digit_capsules = DigitCaps()
            self.decoder = Decoder()

        self.mse_loss = nn.MSELoss()

    def forward(self, data):
        output = self.digit_capsules(self.primary_capsules(self.conv_layer(data)))
        reconstructions, masked = self.decoder(output, data)
        return output, reconstructions, masked

    def loss(self, data, x, target, reconstructions):
        return self.margin_loss(x, target) + self.reconstruction_loss(data, reconstructions)

    def margin_loss(self, x, labels, size_average=True):
        batch_size = x.size(0)

        v_c = torch.sqrt((x ** 2).sum(dim=2, keepdim=True))

        left = F.relu(0.9 - v_c).view(batch_size, -1)
        right = F.relu(v_c - 0.1).view(batch_size, -1)

        loss = labels * left + 0.5 * (1.0 - labels) * right
        loss = loss.sum(dim=1).mean()

        return loss

    def reconstruction_loss(self, data, reconstructions):
        loss = self.mse_loss(reconstructions.view(reconstructions.size(0), -1), data.view(reconstructions.size(0), -1))
        return loss * 0.0005

## 定義訓練函數

In [ ]:
def train(model, optimizer, train_loader, epoch):
    capsule_net = model
    capsule_net.train()
    n_batch = len(list(enumerate(train_loader)))
    total_loss = 0
    for batch_id, (data, target) in enumerate(tqdm(train_loader)):

        target = torch.sparse.torch.eye(10).index_select(dim=0, index=target)
        data, target = Variable(data), Variable(target)
        min_value = torch.tensor([0.0])

        if USE_CUDA:
            data, target = data.cuda(), target.cuda()
            min_value = min_value.cuda()

        optimizer.zero_grad()
        output, reconstructions, masked = capsule_net(data)
        loss = torch.max(capsule_net.loss(data, output, target, reconstructions),min_value)
        loss.backward()
        optimizer.step()
        correct = sum(np.argmax(masked.data.cpu().numpy(), 1) == np.argmax(target.data.cpu().numpy(), 1))
        train_loss = loss.item()
        total_loss += train_loss
        if batch_id % 100 == 0:
            tqdm.write("Epoch: [{}/{}], Batch: [{}/{}], train accuracy: {:.6f}, loss: {:.6f}".format(
                epoch,
                N_EPOCHS,
                batch_id + 1,
                n_batch,
                correct / float(BATCH_SIZE),
                train_loss / float(BATCH_SIZE)
                ))
    tqdm.write('Epoch: [{}/{}], train loss: {:.6f}'.format(epoch,N_EPOCHS,total_loss / len(train_loader.dataset)))

## 定義測試函數

In [ ]:
def test(capsule_net, test_loader, epoch):
    capsule_net.eval()
    test_loss = 0
    correct = 0
    for batch_id, (data, target) in enumerate(test_loader):

        target = torch.sparse.torch.eye(10).index_select(dim=0, index=target)
        data, target = Variable(data), Variable(target)
        min_value = torch.tensor([0.0])

        if USE_CUDA:
            data, target = data.cuda(), target.cuda()
            min_value = min_value.cuda()

        output, reconstructions, masked = capsule_net(data)
        loss = torch.max(capsule_net.loss(data, output, target, reconstructions),min_value)

        test_loss += loss.item()
        correct += sum(np.argmax(masked.data.cpu().numpy(), 1) ==
                       np.argmax(target.data.cpu().numpy(), 1))

    tqdm.write(
        "Epoch: [{}/{}], test accuracy: {:.6f}, loss: {:.6f}".format(epoch, N_EPOCHS, correct / len(test_loader.dataset),
                                                                  test_loss / len(test_loader)))

## 定義參數設定函數

In [ ]:
class Config:
    def __init__(self, dataset='mnist'):
        if dataset == 'mnist':
            # CNN (cnn)
            self.cnn_in_channels = 1
            self.cnn_out_channels = 256
            self.cnn_kernel_size = 9

            # Primary Capsule (pc)
            self.pc_num_capsules = 8
            self.pc_in_channels = 256
            self.pc_out_channels = 32
            self.pc_kernel_size = 9
            self.pc_num_routes = 32 * 6 * 6

            # Digit Capsule (dc)
            self.dc_num_capsules = 10
            self.dc_num_routes = 32 * 6 * 6
            self.dc_in_channels = 8
            self.dc_out_channels = 16

            # Decoder
            self.input_width = 28
            self.input_height = 28

        elif dataset == 'cifar10':
            # CNN (cnn)
            self.cnn_in_channels = 3
            self.cnn_out_channels = 256
            self.cnn_kernel_size = 9

            # Primary Capsule (pc)
            self.pc_num_capsules = 8
            self.pc_in_channels = 256
            self.pc_out_channels = 32
            self.pc_kernel_size = 9
            self.pc_num_routes = 32 * 8 * 8

            # Digit Capsule (dc)
            self.dc_num_capsules = 10
            self.dc_num_routes = 32 * 8 * 8
            self.dc_in_channels = 8
            self.dc_out_channels = 16

            # Decoder
            self.input_width = 32
            self.input_height = 32

        elif dataset == 'your own dataset':
            pass

## 進行訓練與測試

In [ ]:
BATCH_SIZE = 100
N_EPOCHS = 2
LEARNING_RATE = 0.01
MOMENTUM = 0.9

if __name__ == '__main__':
    torch.manual_seed(1)
    # dataset = 'cifar10'
    dataset = 'mnist'
    config = Config(dataset)
    mnist = Dataset(dataset, BATCH_SIZE)

    capsule_net = CapsNet(config)
    capsule_net = torch.nn.DataParallel(capsule_net)
    if USE_CUDA:
        capsule_net = capsule_net.cuda()
    capsule_net = capsule_net.module

    optimizer = torch.optim.Adam(capsule_net.parameters())

    for e in range(1, N_EPOCHS + 1):
        train(capsule_net, optimizer, mnist.train_loader, e)
        test(capsule_net, mnist.test_loader, e)

  0%|          | 1/600 [00:00<06:00,  1.66it/s]

Epoch: [1/2], Batch: [1/600], train accuracy: 0.050000, loss: 0.008995


 17%|█▋        | 101/600 [00:51<04:11,  1.98it/s]

Epoch: [1/2], Batch: [101/600], train accuracy: 0.930000, loss: 0.004105


 34%|███▎      | 201/600 [01:40<03:20,  1.99it/s]

...

Epoch: [2/2], Batch: [401/600], train accuracy: 0.970000, loss: 0.000534


 84%|████████▎ | 501/600 [04:05<00:48,  2.03it/s]

Epoch: [2/2], Batch: [501/600], train accuracy: 0.980000, loss: 0.000429


100%|██████████| 600/600 [04:54<00:00,  2.04it/s]


Epoch: [2/2], train loss: 0.000442
Epoch: [2/2], test accuracy: 0.989000, loss: 0.037293
